## 버트(BERT)

2018년 11월, 구글이 공개한 인공지능 언어 모델 BERT(Bidirectional Encoder Representations from Transformers)는 기존의 단방향 자연어 처리 모델들의 단점을 보완한 양방향 자연어 처리 모델입니다. 검색 문장의 단어를 입력된 순서대로 하나씩 처리하는 것이 아니라, 트랜스포머를 이용하여 구현되었으며 방대한 양의 텍스트 데이터로 사전 훈련된 언어 모델입니다.

![](../Static/587.jpg)

버트의 기본 구조는 그림과 같이 트랜스포머(transformer)라는 인코더를 쌓아 올린 구조로, 주로 `문장 예측(NSP)`을 할 때 사용합니다. 튜닝을 통해 최고의 성능을 낸 기존 사례들을 참고해서 사전 학습된 버트 위에 분류를 위한 신경망을 한층 추가하는 방식을 사용합니다. 즉, 버트는 트랜스포머와 사전 학습을 사용하여 성능을 향상시킨 모델입니다.

버트의 학습 절차는 다음과 같습니다.

1. 그림과 같이 문장을 버트의 입력 형식에 맞게 변환합니다. 이때 문장의 시작은 [CLS], 문장의 끝은 [SEP]로 표시합니다.

2. 한 문장의 단어들에 대해 `토큰화(tokenization)`을 진행합니다. 예를 들어 '고양이'라는 단어의 경우 '고##','#양#','##이'로 토큰화합니다.

3. 마지막으로 각 토큰들에 대해 고유의 아이디를 부여합니다. 토큰이 존재하지 않는 자리는 0으로 채웁니다.

버트 모델은 전이 학습을 기반으로 한다고 했는데, 이떄 전이는 인코더-디코더로 된 모델입니다. 기존 인코더-디코더 모델들과 다르게 CNN,RNN을 이용하지 않고 `어텐션 개념`을 도입했습니다. 즉, 버트에서 전이 학습은 인코더-디코더중 `인코더만 사용`하는 모델입니다.

버트는 두 가지 버전이 있는데, BERT-base(L=12, H=768, A=12)와 BERT-large(L=24, H=1024, A=16)입니다. 이떄 L은 전이 블록 숫자이고, H는 은닉층 크기, A는 전이 블록에서 사용되는 어텐션 블록 숫자입니다. 즉, L,H,A가 크다는 것은 블록을 많이 쌓았고, 표현하는 은닉층이 크며 어텐션 개수를 많이 사용했다는 의미입니다. BERT-base는 학습 파라미터가 1.1억개가 있고, BERT-large는 학습 파라미터가 3.4억개 있습니다.

예제를 진행하기에 앞서 파이토치에서 버트를 사용하기 위한 라이브러리를 설치합니다.

> pip install transformers

> pip install pytorch-transformers

예제에서 사용하는 데이터셋은 네이버 영화 리뷰입니다. 데이터셋은 다음 URL에서도 내려받을 수 있습니다.

https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # 모델 평가를 위해 사용
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
train_df = pd.read_csv('./rating/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./rating/ratings_test.txt', sep='\t')
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
class RatingDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [4]:
train_dataset = RatingDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)

test_dataset = RatingDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=0)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

* 예제에서는 버트 모델 중 'bert-base-uncased'를 사용했지만 한국어를 사용하기 위해서는 'bert-base-multilingual-cased'를 사용하는 것이 맞습니다. 'bert-base-multilingual-cased'의 경우에는 100개 이상의 언어를 적용할 수 있는 모델입니다.

* 'bert-base-uncased' 모델은 버트의 가장 기본적인 모델을 의미하며, `uncased`는 모든 문장을 소문자로 대체하겠다는 것입니다. 또한 `BertTokenizer.from_pretrained`는 사전 훈련된 버트의 토크나이저를 사용하겠다는 의미입니다.

* 데이터를 분류하기 위해 버트 모델을 내려받습니다. 토크나이저처럼 사전 훈련된 버트 모델을 명시해야 합니다. 즉, 버트 모델을 생성하는 단계입니다.

다음은 최적화된 모델을 저장하기 위한 함수입니다.

In [6]:
import os

def save_checkpoint(save_path, model, valid_loss=None): # 모델 평가를 위해 훈련 과정을 저장
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    state_dict = {'model_state_dict' : model.state_dict(), 'valid_loss' : valid_loss}
    torch.save(state_dict, save_path)
    print(f'Model saved to => {save_path}')

def load_checkpoint(load_path, model): # save_checkpoint 함수에서 저장된 모델을 가져옵니다.
    if load_path == None:
        print('경로가 정확하지 않습니다. {load_path}')
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <= {load_path}')
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

def save_metrics(save_path, train_loss_list, global_steps_list): # 훈련에 대한 오차와 에포크를 저장
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    state_dict = {'train_loss_list' : train_loss_list, 'global_steps_list' : global_steps_list}
    torch.save(state_dict, save_path)
    print(f'Model saved to => {save_path}')

def load_metrics(load_path): # save_metrics에 저장해 둔 정보를 불러옵니다.
    if load_path == None:
        print('경로가 정확하지 않습니다. {load_path}')
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <= {load_path}')
    return state_dict['train_loss_list'], state_dict('global_steos)list')


버트 모델을 학습시키기 위한 함수를 정의합니다. 코드가 상당히 길어 보이지만 복잡하지는 않습니다. 옵티마이저 설정, 에포크 주기별로 오차를 기록하는 것이 학습 과정의 전부입니다. 단지 훈련이 하나의 함수에 포함되어 있어 길어 보일 뿐입니다.

In [11]:
def train(model, optimizer, criterion=nn.BCELoss(), num_epochs=5, eval_every=len(train_loader)//2, best_train_loss=float('Inf')): # 영화 리뷰는 좋고 나쁨만 있으므로 바이너리 크로스 엔트로피를 사용
    total_correct = 0.0
    total_len = 0.0
    running_loss = 0.0
    global_step = 0
    train_loss_list = []
    global_steps_list = []

    model.train() # 모델 훈련


    for epoch in range(num_epochs):
        for text, label in train_loader:
            optimizer.zero_grad()
            encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
            padded_list = [e + [0] * (512-len(e)) for e in encoded_list]
            sample = torch.tensor(padded_list)
            sample, label = sample.to(device), label.to(device)
            labels = torch.tensor(label)
            outputs = model(sample, labels=labels)
            loss, logits = outputs

            pred = torch.argmax(F.softmax(logits), dim=0)
            correct = pred.eq(labels)
            total_correct += correct.sum().item()
            total_len += len(labels)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            global_step += 1

    save_metrics('./rating/result/metrics.pt', train_loss_list, global_steps_list)
    save_checkpoint('./rating/result/model.pt', model=model)
    print('훈련종료')


* `torch.argmax`는 출력된 열 중에서 `가장 큰 값을 반환`합니다. 예를 들어 다음과 같이 사용할 수 있습니다.
```py
import torch
a = torch.randn(4, 4)
argmax = torch.argmax(a)
print(a)
print(argmax)
```
다음과 같이 4x4 형태의 임의의 텐서를 생성한 후 그중 가장 큰 값(2.2177)을 갖는 인덱스(13)를 반환합니다.
```py
tensor([[ 0.6908, -0.2365, 0.3776, -0.7609],
        [ 1.0446, -0.2636, 0.4735,  1.6540],
        [-1.4138,  0.3427, 0.4632, -0.4743],
        [-0.2170,  2.2177, 0.2166, -0.2508]])
tensor(13)
```

따라서 `softmax(logits)` 값 중에서 가장 큰 값을 반환합니다.

* if best_valid_loss > average_valid_loss:
    * 데이터셋에 대한 오차가 감소할 때마다 모델을 저장하여 가장 낮은 오차를 갖는 모델로 학습을 마치도록 합니다.

사전 학습된 버트 모델에서 파라미터(옵티마이저와 학습률)을 미세 조정 후 모델을 학습시킵니다.

In [12]:
optimizer = optim.Adam(model.parameters(), lr=2e-5)
train(model=model, optimizer=optimizer)

C:\Users\aqs45\AppData\Local\Temp\ipykernel_20388\3155855292.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 4.00 GiB total capacity; 3.33 GiB already allocated; 0 bytes free; 3.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF